In [77]:
import cv2
import numpy as np
from PIL import Image
import dlib
import face_recognition
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [78]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [79]:
def img_list():
    format_img = ['.png']
    final = []
    for i in format_img:
        image_list = glob.glob("data/Want to find/Images/unnatural_death_images/cleaned/*"+i)
        for j in image_list:
            final.append(j)
    return final
        
image_list = img_list()

In [80]:
len(image_list)

3926

In [81]:
temp_shape_list = []
img_list_temp = []
for img in image_list:
    image = face_recognition.load_image_file(img)
   
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 0)
    if len(rects) > 0:
        for rect in rects:
            x = rect.left()
            y = rect.top()
            w = rect.right()
            h = rect.bottom()
            shape = predictor(gray, rect)
            img_list_temp.append(img)
            temp_shape_list.append(shape)

In [82]:
len(img_list_temp)

1025

In [83]:
len(temp_shape_list)

1025

In [84]:
def shape_to_normal(shape):
    shape_normal = []
    for i in range(0, 5):
        shape_normal.append((i, (shape.part(i).x, shape.part(i).y)))
    return shape_normal
def get_eyes_nose_dlib(shape):
    nose = shape[4][1]
    left_eye_x = int(shape[3][1][0] + shape[2][1][0]) // 2
    left_eye_y = int(shape[3][1][1] + shape[2][1][1]) // 2
    right_eyes_x = int(shape[1][1][0] + shape[0][1][0]) // 2
    right_eyes_y = int(shape[1][1][1] + shape[0][1][1]) // 2
    return nose, (left_eye_x, left_eye_y), (right_eyes_x, right_eyes_y)

In [85]:
def distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [86]:
def cosine_formula(length_line1, length_line2, length_line3):
    cos_a = -(length_line3 ** 2 - length_line2 ** 2 - length_line1 ** 2) / (2 * length_line2 * length_line1)
    return cos_a

In [87]:
def rotate_point(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy


def is_between(point1, point2, point3, extra_point):
    c1 = (point2[0] - point1[0]) * (extra_point[1] - point1[1]) - (point2[1] - point1[1]) * (extra_point[0] - point1[0])
    c2 = (point3[0] - point2[0]) * (extra_point[1] - point2[1]) - (point3[1] - point2[1]) * (extra_point[0] - point2[0])
    c3 = (point1[0] - point3[0]) * (extra_point[1] - point3[1]) - (point1[1] - point3[1]) * (extra_point[0] - point3[0])
    if (c1 < 0 and c2 < 0 and c3 < 0) or (c1 > 0 and c2 > 0 and c3 > 0):
        return True
    else:
        return False

In [88]:
noses = []
left = []
right = []
for (s, img, i) in zip(temp_shape_list, img_list_temp, range(0, len(img_list_temp))):
    shape = shape_to_normal(s)
    nose, left_eye, right_eye = get_eyes_nose_dlib(shape)
    noses.append(nose), left.append(left_eye), right.append(right_eye)
#     center_of_forehead = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
#     center_pred = (int((x + w) / 2), int((y + y) / 2))
#     length_line1 = distance(center_of_forehead, nose)
#     length_line2 = distance(center_pred, nose)
#     length_line3 = distance(center_pred, center_of_forehead)
#     cos_a = cosine_formula(length_line1, length_line2, length_line3)
#     angle = np.arccos(cos_a)
#     rotated_point = rotate_point(nose, center_of_forehead, angle)
#     rotated_point = (int(rotated_point[0]), int(rotated_point[1]))
#     if is_between(nose, center_of_forehead, center_pred, rotated_point):
#         angle = np.degrees(-angle)
#     else:
#         angle = np.degrees(angle)
#     img = Image.fromarray(img)
#     img = np.array(img.rotate(angle))
#     cv2.imwrite('realigning/'+str(i)+'.jpg', img)
#     print(nose, left_eye, right_eye)

In [18]:
!pip install Image

In [89]:

import sys, math
from PIL import Image

def Distance(p1,p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return math.sqrt(dx*dx+dy*dy)

def ScaleRotateTranslate(image, angle, center = None, new_center = None, scale = None, resample=Image.BICUBIC):
    if (scale is None) and (center is None):
        return image.rotate(angle=angle, resample=resample)
    nx,ny = x,y = center
    sx=sy=1.0
    if new_center:
        (nx,ny) = new_center
    if scale:
        (sx,sy) = (scale, scale)
    cosine = math.cos(angle)
    sine = math.sin(angle)
    a = cosine/sx
    b = sine/sx
    c = x-nx*a-ny*b
    d = -sine/sy
    e = cosine/sy
    f = y-nx*d-ny*e
    return image.transform(image.size, Image.AFFINE, (a,b,c,d,e,f), resample=resample)

def CropFace(image, eye_left=(0,0), eye_right=(0,0), offset_pct=(0.2,0.2), dest_sz = (70,70)):
  # calculate offsets in original image
    offset_h = math.floor(float(offset_pct[0])*dest_sz[0])
    offset_v = math.floor(float(offset_pct[1])*dest_sz[1])
    # get the direction
    eye_direction = (eye_right[0] - eye_left[0], eye_right[1] - eye_left[1])
    # calc rotation angle in radians
    rotation = -math.atan2(float(eye_direction[1]),float(eye_direction[0]))
    # distance between them
    dist = Distance(eye_left, eye_right)
    # calculate the reference eye-width
    reference = dest_sz[0] - 2.0*offset_h
    # scale factor
    scale = float(dist)/float(reference)
    # rotate original around the left eye
    image = ScaleRotateTranslate(image, center=eye_left, angle=rotation)
    # crop the rotated image
    crop_xy = (eye_left[0] - scale*offset_h, eye_left[1] - scale*offset_v)
    crop_size = (dest_sz[0]*scale, dest_sz[1]*scale)
    image = image.crop((int(crop_xy[0]), int(crop_xy[1]), int(crop_xy[0]+crop_size[0]), int(crop_xy[1]+crop_size[1])))
    # resize it
    image = image.resize(dest_sz, Image.ANTIALIAS)
    return image


In [90]:
save_path = "data/Want to find/Images/unnatural_death_images/realigned/"

In [91]:
for (img, le, re, sp) in zip(img_list_temp, left, right, range(len(image_list))):
    img_o = Image.open(img)
    CropFace(img_o, eye_left=le, eye_right=re, offset_pct=(0.42,0.42), dest_sz=(150,150)).save(save_path+str(sp)+'.png')
    
    